In [383]:
import csv
import pandas as pd
import random
from ast import literal_eval
from itertools import chain
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import re
from collections import OrderedDict
import inflect
p = inflect.engine()

wnl = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


In [384]:
Toxical = pd.read_csv("tsd_trial.csv")
SwearWords = pd.read_csv("SwearWords.csv")
SwearWordsList = SwearWords['SWEAR'].to_list() #.replace("-$", "", regex=True)
replacements=[('\xa0-', ''), ('\\\\', '')] 
for pat, repl in replacements:
    SwearWordsList1 = list(OrderedDict.fromkeys([stemmer.stem(wnl.lemmatize(re.sub(pat, repl, i.lower()))) for i in SwearWordsList] + [re.sub(pat, repl, i.lower()) for i in SwearWordsList]))
SwearWordsList2 = list(OrderedDict.fromkeys([p.plural(word.lower()) for word in SwearWordsList1] + [p.singular(word.lower()) for word in SwearWordsList1]))
SwearWordsList = [w for w in SwearWordsList2 if not w in stop_words]
Toxical.head()

AttributeError: 

In [375]:
def checkToxicalWords (sampledText, index):
    toxicalWords = []
    charList = list(sampledText[int(i)] for i in literal_eval(Toxical.iloc[index][0])) #Extract character from there
    if ' ' in charList:
        splitList = [i for i,x in enumerate(charList) if x == ' '] #Get the index of " " as splitter list
        temp = zip(chain([0], splitList), chain(splitList, [None])) #Split the list by " "
        res = list(''.join(charList[i : j]) for i, j in temp)
        if len(res) > 0:
            resMod = [res[0]] + [i[1:] for i in res[1:]] #Remove the starting " " for the words
#                 print(resMod) 
        toxicalWords = resMod
    else:
        toxicalWords = [str(''.join(charList))]
#     toxicalWords =  [i for i in toxicalWords] 
    return toxicalWords

In [369]:
# sampledRow = random.randint(0, len(Toxical) - 1) #Sample a random row
sampledRow = 2
sampledText = Toxical.iloc[sampledRow][1] #Extract document text from that row

checkToxicalWords (sampledText, sampledRow)

['idiots']

In [376]:
Toxical['textSplit'] = [i.split() for i in Toxical['text']]
# sampledRow = random.randint(0, len(Toxical) - 1) #Sample a random row
# sampledText = Toxical.iloc[sampledRow][1] #Extract document text from that row
def getSwearIndex (sampledText, index):
    indexList = []
    intersection = set(sampledText.split()).intersection(SwearWordsList)
    toxicalWordsLabeled = checkToxicalWords (sampledText, index)
    toxicalWordsDetect = list(intersection)
    if len(intersection) > 0:
        for i in intersection: 
            indexList.append([item for item in range(sampledText.find(i), sampledText.find(i) + len(i))])
        indexList = [val for sublist in indexList for val in sublist]
    return toxicalWordsLabeled, toxicalWordsDetect, indexList

def generateNewColumns (dataFrame):
    dataFrame['labeled_toxical_words'] = ''
    dataFrame['candidate_toxical_words'] = ''
    dataFrame['candidate_span'] = ''
    for index, row in dataFrame.iterrows():
        row['labeled_toxical_words'], row['candidate_toxical_words'], row['candidate_span'] = getSwearIndex(row['text'].lower(), index)
    return dataFrame

In [385]:
ToxicalMatched = generateNewColumns(Toxical)
ToxicalMatched[["candidate_span"]]

,candidate_span
0,[]
1,[]
2,[]
3,"[87, 88, 89, 90, 91, 92]"
4,[]
...,...
685,[]
686,[]
687,[]
688,[]


In [387]:
ToxicalMatched[["candidate_span"]].to_csv("originalSubmission.txt", sep='\t')